# **Coronavirus (COVID-19) in India**


> India reported the first confirmed case of the coronavirus infection on 30 January 2020 in the state of Kerala. The affected had a travel history from Wuhan, China. Since the Wuhan coronavirus outbreak in China, the first case of coronavirus in India, the second most populated country in the world, was reported in the state of Kerala. New confirmed cases are being reported in multiple cities such as New Delhi, Mumbai, Bengaluru, Hyderabad, and Patna.

Despite a strong response at the outset of the pandemic, as of Sept 22, India has the world's fastest growing outbreak of COVID-19 in absolute numbers according to WHO, reporting more than 5·6 million infections. Restrictions began to be lifted in June, and this relaxation has continued in the face of a continuing dramatic increase in case numbers nationally. Beneath these alarming national figures, the pattern of spread in India is nuanced and complex, with marked differences between states, and between rural and urban areas. For example, cities like Kolkata and rural areas in the north of India were relatively spared the outbreak initially, whereas Delhi, with strong international connections, was at the forefront of the first wave. Even so, India is clearly facing a dangerous period. The country has responded well in many regards, especially for such a large and diverse nation. India instigated a national lockdown in March, which was praised by WHO. During the lockdown period, tertiary care provision was increased, including access to specialist equipment such as ventilators. Testing numbers also increased quickly, with India being among the first to roll out innovations like pooled testing. India has also been at the forefront of efforts to develop and manufacture a vaccine, both through domestic vaccine candidates and manufacturers such as the Serum Institute of India preparing production capacity for internationally developed vaccine candidates.

![pic](https://thefederal.com/file/2020/06/Coronavirus-India.jpg)


## This Notebook Demonstrates The Following:


* Visualizing Covid-19 data in different ways.
* Plotting data on India map using shapefile and geopandas library.
* An interactive Time slider choropleth map as follows (Zoom on India before using slider bar).

# <span style="color:blue">Please Upvote If You Like, Use Or Learn From This.

## Import Libraries of Python

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Loading the data files

In [ ]:
district_data = pd.read_csv("/kaggle/input/covid19-corona-virus-india-dataset/district_level_latest.csv")
state_data = pd.read_csv("/kaggle/input/covid19-corona-virus-india-dataset/state_level_latest.csv")
country_data = pd.read_csv("/kaggle/input/covid19-corona-virus-india-dataset/nation_level_daily.csv")
patient_data = pd.read_csv("/kaggle/input/covid19-corona-virus-india-dataset/patients_data.csv")
tests_state_data = pd.read_csv("/kaggle/input/covid19-corona-virus-india-dataset/tests_state_wise.csv")

## Removing Unnecessary Columns From District Data

In [ ]:
district_data = district_data.drop(['District_Notes','Last_Updated'],axis=1)
district_data.info()

# Plotting the important features across the states

In [ ]:
# Drop the column of Totals
state_data = state_data.drop(0)

# Plot the Confirmed, Recovered, Active and Deaths on the same plot using matplotlib
plt.figure(figsize=(12,10))
x = state_data['State_code']
y = state_data['Confirmed']
plt.plot(x,y,marker='o',label="Confirmed")
y = state_data['Recovered']
plt.plot(x,y,marker='o',label="Recovered")
y = state_data['Active']
plt.plot(x,y,marker='o',label="Active")
y = state_data['Deaths']
plt.plot(x,y,marker='o',label="Deaths")
plt.legend();

### Selecting The Data For Top 20 States wrt 'Confirmed' Cases.

In [ ]:
state_top20 = state_data.nlargest(20,'Confirmed')
state_top20

### Drawing Bar Plots For The Top 20 States 

In [ ]:
# Position of bars on x-axis
ind = np.arange(20)

# Width of a bar 
width = 0.4

plt.figure(figsize=(15,12))
x = state_top20['State_code']
y = state_top20['Confirmed']
plt.bar(ind+width/2,y,align='edge',width=width,label="Confirmed")
y = state_top20['Recovered']
plt.bar(ind+width,y,align='edge',width=width,label="Recovered")
y = state_top20['Active']
plt.bar(ind+3*width/2,y,align='edge',width=width,label="Active")
y = state_top20['Deaths']
plt.bar(ind+2*width,y,align='edge',width=width,label="Deaths")

plt.xticks(ind + 3*width/2, x)
plt.legend();


### Selecting Data For Top 10 Districts From India wrt 'Confirmed' Cases

In [ ]:
district_top10 = district_data.nlargest(10,'Confirmed')
district_top10

### Drawing Bar Plots For Top 10 Districts

In [ ]:
# Position of bars on x-axis
ind = np.arange(10)

# Width of a bar 
width = 0.4

plt.figure(figsize=(15,12))
x = district_top10['District']
y = district_top10['Confirmed']
plt.bar(ind+width/2,y,align='edge',width=width,label="Confirmed")
y = district_top10['Recovered']
plt.bar(ind+width,y,align='edge',width=width,label="Recovered")
y = district_top10['Active']
plt.bar(ind+3*width/2,y,align='edge',width=width,label="Active")
y = district_top10['Deceased']
plt.bar(ind+2*width,y,align='edge',width=width,label="Deseased")

plt.xticks(ind + 3*width/2, x)
plt.legend()

### Testing Dataset Processing To Plot

In [ ]:
tests = tests_state_data.loc[tests_state_data['Updated On'] == '06/08/2020']
tests = tests[['State','Total Tested','Positive']]
tests = tests.dropna()
tests.info()

### Drawing An Interactive Map To Visualize The Relationship Of Number Of Tests With Positive (Confirmed) Cases Using Plotly

In [ ]:
import plotly.express as px

fig = px.scatter(tests, x="Total Tested", y="Positive", text="State", log_x=True, log_y=True, size_max=100, color="Positive")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Life Expectency', title_x=0.5)
fig.show() 
# Hover over the image to see the details of each state

# Plotting Covid-19 data on India Map using geopandas: 
### First read the shape file of India which has the geometries of Indian states.

In [ ]:
import geopandas as gpd

fp = r'../input/indiageofiles/india-polygon.shp'
map_df = gpd.read_file(fp)
map_df.rename(columns={'st_nm': 'State'},inplace=True)
map_df.head() #check the head of the file

## Merging The State Data File With Shape File

In [ ]:
data_merge = map_df.merge(state_data, on = 'State', how = 'left')
data_merge.head()

### Plotting 'Confirmed' Cases On Map of India : Heatmap

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
ax.set_title('Statewise Confirmed Cases', fontdict={'fontsize': '25', 'fontweight' : '10'})

# plot the figure
data_merge.plot(column='Confirmed',cmap='YlOrRd', linewidth=0.8, ax=ax, edgecolor='0', legend=True,markersize=[39.739192, -104.990337]);

### Plotting 'Recovered' Cases On Map of India : Heatmap

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
ax.set_title('Statewise Recovered Cases', fontdict={'fontsize': '25', 'fontweight' : '10'})
data_merge.plot(column='Recovered',cmap='Greens', linewidth=0.8, ax=ax, edgecolor='0', legend=True,markersize=[39.739192, -104.990337]);


### Plotting 'Deaths' Cases On Map Of India : Heatmap

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))
ax.axis('off')
ax.set_title('Statewise Deaths', fontdict={'fontsize': '25', 'fontweight' : '10'})
data_merge.plot(column='Deaths',cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0', legend=True,markersize=[39.739192, -104.990337]);

# Interactive TimeSlider Choropleth map
A choropleth map is a type of thematic map in which areas are shaded or patterned in proportion to a statistical variable that represents an aggregate summary of a geographic characteristic within each area. We map the Covid-19 summary on India map.


### Importing The Libraries And Reading Daily Covid-19 Data Of Indian States

In [ ]:
import branca.colormap as cm
import folium
states_daily_data = pd.read_csv("/kaggle/input/covid19-corona-virus-india-dataset/state_level_daily.csv")
states_daily_data.head()

### We drop the unnecessary columns, rename the columns of state names in both the  states data and map_df (shapefile) data and then merge the two data according to 'State'.

In [ ]:
states =states_daily_data.drop(['State'],axis=1)
states = states.rename(columns={'State_Name': 'State'})
states_map = states.merge(map_df,on='State')
states_map.head()

### Deleting the rows containing confirmed cases equal to zero

In [ ]:
states_df = states_map[states_map.Confirmed != 0]
states_df.info()

### Transforming confirmed cases to log10 scale to reduce the scale of plotting.

In [ ]:
states_df['log_Confirmed'] = np.log10(states_df['Confirmed'])

In [ ]:
states_df.head()

### Convert Date Into Standard Format

In [ ]:
states_df['date'] = pd.to_datetime(states_df['Date']).astype(int) / 10**9
states_df['date'] = states_df['date'].astype(int).astype(str)

In [ ]:
states_df.head()

### Selecting Important Columns For Plotting

In [ ]:
states_df = states_df[['State', 'date', 'log_Confirmed', 'geometry']]
states_df = states_df.dropna() #Drop all the rows with any null values
states_df.info()

### Sorting The Data According To Date and State Names

In [ ]:
states_df = states_df.sort_values(['State','date']).reset_index(drop=True)
states_df.head()

### Defining a colour map in terms of the log of the number of confirmed cases

In [ ]:
max_colour = max(states_df['log_Confirmed'])
min_colour = min(states_df['log_Confirmed'])
cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
states_df['colour'] = states_df['log_Confirmed'].map(cmap)

### Next, we create style dictionary 

In [ ]:
states_list = states_df['State'].unique().tolist()
states_idx = range(len(states_list))

style_dict = {}
for i in states_idx:
    states = states_list[i]
    result = states_df[states_df['State'] == states]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['date']] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

### Make a dataframe containing the features for each Indian state

In [ ]:
states_geom_df = states_df[['geometry']]
states_geom_gdf = gpd.GeoDataFrame(states_geom_df)
states_geom_gdf = states_geom_gdf.drop_duplicates().reset_index()

## Final step : create our map and add a colourbar. Save the choropleth map created in HTML format so that you can visualize it in the browser.

In [ ]:
from folium.plugins import TimeSliderChoropleth

slider_map = folium.Map(zoom_start=4,location=[21, 78])

_ = TimeSliderChoropleth(
    data=states_geom_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)

_ = cmap.add_to(slider_map)
cmap.caption = "Log of number of confirmed cases"
slider_map.save(outfile='Covid19_Map_India.html')


## View HTML output file in Notebook

In [ ]:
from IPython.display import IFrame
IFrame(src='./Covid19_Map_India.html', width=700, height=600)

#Slide the bar to see the Covid-19 spread across Indian states from 14 March'20 to 6 August'20

# <span style="color:blue">**Please Upvote If You Like, Use Or Learn From This.**

# <span style="color:blue">**Also Comment For Improvisation Of The Notebook.**


# <span style="color:blue">**Thank You!!!**

